In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
print('Packages installed. Go gettem.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    attrs-20.1.0               |     pyh9f0ad1d_0          41 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-3.1           |   py36h45558ae

In [4]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import requests

In [11]:

print('lxml is installed I promise not to ask you to install it again')

lxml is installed I promise not to ask you to install it again


In [12]:
df_euro_cities_wiki=pd.read_html("https://en.wikipedia.org/wiki/List_of_European_cities_by_population_within_city_limits")[0]

wiki_dfdrop=df_euro_cities_wiki.drop(columns=['Unnamed: 0', 'Date', '2011 Eurostatpopulation[1]', 'Image', 'Ref.'])

wiki_dfdrop["City"][0] = "Istanbul"
wiki_dfdrop["City"][1] = "Moscow"
wiki_dfdrop["Officialpopulation"][0] = "15519267"
wiki_dfdrop=wiki_dfdrop.rename(columns={'Officialpopulation': 'Official Population'})

df_bigcities=wiki_dfdrop.head(15)

discard = []
keep = []

for row in df_bigcities['Location']:
    try:
        discard.append(row.split('/')[0])
        keep.append(row.split('/')[1])
    except:
        print("Error during splitting")
        
keep_df=pd.DataFrame(keep,columns=['locat'])

lat=[]
lon=[]

for row in keep_df['locat']:
    try:
        lat.append(row.split('N')[0])
        lon.append(row.split('N')[1])
    except:
        print('No Joy')

df_bigcities['Latitude']=lat
df_bigcities['Longitude']=lon

ImportError: lxml not found, please install it

In [1]:


df_bigcities['Latitude'] = df_bigcities['Latitude'].str.replace('°','')
df_bigcities['Longitude'] = df_bigcities['Longitude'].str.replace('°E','')
df_bigcities['Latitude'] = df_bigcities['Latitude'].str.replace(' ','')
df_bigcities['Latitude'] = df_bigcities['Latitude'].str.replace('\ufeff','')
df_bigcities['Longitude'] = df_bigcities['Longitude'].str.replace(' ','')
df_bigcities['Longitude'][2] = "-0.1275"
df_bigcities['Longitude'][5] = "-3.716667"
df_bigcities=df_bigcities.drop(columns=['Location'])



NameError: name 'df_bigcities' is not defined

In [ ]:
df_bigcities['Latitude']=df_bigcities.Latitude.astype(float)
df_bigcities['Longitude']=df_bigcities.Longitude.astype(float)
df_bigcities.dtypes



In [ ]:
map_bigcities = folium.Map(location=[54.54, 25.19], zoom_start=4)
for lat, lng, city, country in zip(df_bigcities['Latitude'], df_bigcities['Longitude'], df_bigcities['City'], df_bigcities['Country']):
    label = '{}, {}'.format(city, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bigcities) 

map_bigcities

In [ ]:
CLIENT_ID = 'QRY31Y2XVZC3RCNJEX3FUROCNZ3VX5X15GAXR3STYVN1SERH' 
CLIENT_SECRET = 'ROFNZQUDFCHYJJBNPM00MHYNBRKCXEU33DLIYTT2ZBRN2I5U'
VERSION = '20180605'

LIMIT = 10000
Radius = 250
def getNearbyVenues(names, latitudes, longitudes, radius=250):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse&sortByPopularity=1'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
            
        

        results = requests.get(url).json()["response"]['groups'][0]['items']        
        
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue',          
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category']
    
    return(nearby_venues)

venueseu = getNearbyVenues(names=df_bigcities['City'],
                                   latitudes=df_bigcities['Latitude'],
                                   longitudes=df_bigcities['Longitude'])



In [ ]:
venues_onehot = pd.get_dummies(venueseu[['Venue Category']], prefix="", prefix_sep="")
venues_onehot['City'] = venueseu['City'] 
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

venues_grouped = venues_onehot.groupby('City').mean().reset_index()

num_top_venues = 5

for city in venues_grouped['City']:
    print("----"+city+"----")
    temp = venues_grouped[venues_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['City'] =  venues_grouped['City']

for ind in np.arange(venues_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

cities_venues_sorted.head()

